In [1]:
import numpy as np

In [15]:
'''
- expected input volume to be 3d array
- filters is a list of 3d array filters
- biases is a list of bias terms, one for each filter
'''
def convolution(input_volume, filters, biases, stride=1, zero_padding=0):
    
    # constrain to square images
    num_channels, _, img_dim_orig = input_volume.shape
    num_filters, _, __, filter_dim = filters.shape
    
    
    # shape
    # np.ones(shape=(2, 2, 3, 3))
    # num filt, depth, num rows, num cols
    
    # zero padding adds zeroes around the input, but not along the depth dimension
    image = np.zeros(shape=(num_channels, img_dim_orig + 2 * zero_padding, img_dim_orig + 2 * zero_padding))
    image[:, zero_padding:-zero_padding, zero_padding:-zero_padding] = input_volume
    
    return image
        